In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

BubbleChart 富集

In [6]:
def BubbleChart(df, color, bubble_num, bubble_size, go_type='all', width=1000, height=800):

    # 拷贝并重命名列名
    df.columns = ["Class", "ID", "Description", "Counts", "Enrichment Score", "P.adjust"]
    # df.columns = ["ID", "Description", "Counts", "Enrichment Score", "Class"]

    df = df.sort_values(by='Counts', ascending=False)

    # 选取p值小于0.05的数据
    df = df[df["P.adjust"] < 0.05]
    
    # df只保留前num行数据
    df = df.iloc[:bubble_num]

    if go_type == "BP":
        df = df[df["Class"].str.contains("Biological process")]
    elif go_type == "CC":
        df = df[df["Class"].str.contains("Cellular component")]
    elif go_type == "MF":
        df = df[df["Class"].str.contains("Molecular function")]
    else:
        pass

    # 创建图表
    fig = px.scatter(
        df,
        x="Enrichment Score",
        y="Description",
        size="Counts",
        color="P.adjust",
        color_continuous_scale=color,  # _r表示颜色反转
        # color_continuous_scale='Geyser_r',
        opacity=0.85,
        hover_name="Class",
        hover_data=["ID", "Description", "Counts", "Enrichment Score", "P.adjust"],
        size_max=bubble_size,
    )

    # 设置图表属性
    fig.update_layout(
        title="GO Enrichment Analysis",
        xaxis_title = "Enrichment Score",
        yaxis_title = "Description",
        yaxis = dict(autorange="reversed"),  # y轴反转
        font=dict(family="Arial", size=14),
        template="plotly_white",
        # template="simple_white",
        width=width,  # 图表宽度
        height=height,  # 图表高度
    )

    # 设置颜色轴的属性
    colorbar_title = "P.adjust"
    colorbar_tickformat = ".3f"
    colorbar_dtick = 0.005
    fig.update_coloraxes(
        colorbar_title=colorbar_title,
        colorbar_tickformat=colorbar_tickformat,
        colorbar=dict(dtick=colorbar_dtick),
    )
    
    # 显示图表并返回
    return fig

In [4]:
df_go_enrich = pd.read_csv("./go_enrich.tsv", sep="\t")
# 指定列
df_go_enrich = df_go_enrich[["GO_ID", "GO_Function", "Counts", "Enrichment Score", "P.adjust"]]
df_go_enrich.head()

,ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count,category
0,GO:0003924,GTPase activity,24/646,71/6066,1.231164e-07,0.000013,0.000011,ANI_1_1776014/ANI_1_352154/ANI_1_124/ANI_1_250...,24,MF
1,GO:0016887,ATPase activity,31/646,111/6066,2.652500e-07,0.000014,0.000012,ANI_1_1902094/ANI_1_2222094/ANI_1_1288084/ANI_...,31,MF
2,GO:0005525,GTP binding,26/646,88/6066,7.525797e-07,0.000026,0.000022,ANI_1_1776014/ANI_1_352154/ANI_1_124/ANI_1_250...,26,MF
3,GO:0003676,nucleic acid binding,45/646,231/6066,3.451899e-05,0.000906,0.000763,ANI_1_430184/ANI_1_2142064/ANI_1_2152074/ANI_1...,45,MF
4,GO:0004842,ubiquitin-protein transferase activity,9/646,19/6066,5.753337e-05,0.001208,0.001017,ANI_1_212054/ANI_1_168134/ANI_1_386014/ANI_1_2...,9,MF


In [8]:
df_GoEnrich = pd.read_csv("GO_Enrichment_demo.csv")
df_BubbleChart = df_GoEnrich[['Class', 'GO_ID', 'GO_Name', 'HitsGenesCountsInSelectedSet', 'EnrichmentScore', 'corrected p-value(BH method)']]
df_GoEnrich.head()

,Class,GO_Name,GO_ID,GO_Level,P_value,EnrichmentScore,HitsGenesCountsInSelectedSet,HitsGenesCountsInBackground,AllGenesCountsInSelectedSet,AllGenesCountsInBackground,GenesOfSelectedSetInGOterm,corrected p-value(BH method)
0,Molecular function,RNA binding,GO:0003723,5,4.330000e-15,3.996168,37,89,258,2480,"rplR,rplT,rplN,rplW,rplY,rplB,rplE,srmB,prfA,r...",2.650000e-12
1,Molecular function,structural constituent of ribosome,GO:0003735,3,2.210000e-14,4.893587,28,55,258,2480,"rpmG,rplR,rplU,rplT,rplN,rplW,rplY,rplC,rplB,r...",6.750000e-12
2,Molecular function,structural molecule activity,GO:0005198,2,6.360000e-13,4.412251,28,61,258,2480,"rpmG,rplR,rplU,rplT,rplN,rplW,rplY,rplC,rplB,r...",1.290000e-10
3,Molecular function,binding,GO:0005488,2,1.820000e-10,1.348980,183,1304,258,2480,"minE,arcA,msrB,nusA,sspA,yqjD,hmp,fabI,wrbA,ac...",2.770000e-08
4,Molecular function,"ligase activity, forming carbon-oxygen bonds",GO:0016875,4,7.440000e-10,6.408269,14,21,258,2480,"gltX,argS,glyS,pheT,ileS,trpS,cysS,hisS,tyrS,a...",9.090000e-08


In [9]:
BubbleChart(df_BubbleChart, 'Rdbu_r', 50, 30, width=1000, height=600)

ValueError: Length mismatch: Expected axis has 6 elements, new values have 5 elements